In [2]:
import pandas as pd
import os 
import requests
import threading


excel_path = r"C:\Users\jarno\Downloads\kmo's_Vlaanderen_2021.xlsx"
onedrive_path = r"C:\Users\jarno\OneDrive - Hogeschool Gent\Data_engineering2"
nieuwe_onedrive_path = r"C:\Users\jarno\OneDrive - Hogeschool Gent\Data_engineering2\shared"
df = pd.read_excel(excel_path, sheet_name="Lijst")



In [ ]:
ondernemings_nummers = df["Ondernemingsnummer"].tolist()
ondernemings_nummers = [x.replace(" ", "") for x in ondernemings_nummers]
# ondernemings_nummers_in_onedrive = [x.split("_")[1] for x in os.listdir(onedrive_path) if x != "shared"]
ondernemings_nummers_in_onedrive_shared = [x.split("_")[1] for x in os.listdir(nieuwe_onedrive_path)]

shared_numbers = set(ondernemings_nummers).intersection(ondernemings_nummers_in_onedrive_shared)
unique_numbers = set(ondernemings_nummers).difference(ondernemings_nummers_in_onedrive_shared)

print(len(ondernemings_nummers))
print(len(ondernemings_nummers_in_onedrive_shared))
print(unique_numbers)


In [ ]:
# move all shared numbers to a new directory
# ! niet meer runnen
for file in os.listdir(onedrive_path):
    if file.split("_")[1] in shared_numbers:
        os.rename(os.path.join(onedrive_path, file), os.path.join(onedrive_path, "shared", file))

In [ ]:
websites = df["Web adres"].tolist()
websites = [x for x in websites if str(x) != "nan"]
# write all websites to a txt file 
with open("websites.txt", "w") as f:
    for website in websites:
        f.write(website + "\n")

websites = ["http://" + x if "http://" not in x else x for x in websites]

def check_website(website):
    try:
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}

        response = requests.get(website, headers=headers)
        print(response.status_code)
        if response.status_code != 200:
            with open("not_online.txt", "a") as f:
                f.write(response.status_code + "  " + website + "\n")
    except Exception as e:
        with open("not_online.txt", "a") as f:
            f.write(website + " " + str(e) + "\n")

threads = []
for website in websites:
    t = threading.Thread(target=check_website, args=(website,))
    threads.append(t)
    t.start()
    print(len([x for x in threads if x.is_alive()]))

# 900 sites kon ik niet aan, main reason lijkt te zijn omdat we geredirect worden, dus mss programma schrijven dat redirects volgt en kijke hoeveel sites er dan nog zijn 
# Ook checke als http:// niet werkt, dan probeer ik https:// 


In [ ]:

with open("../data/links_using_adres.txt", "r") as f:
    data = f.read().split("\n")
    nrs = [x.split(":")[1] for x in data]

ondernemings_nummers
